## Building a SMS Spam Filter algorith with Naive Bayes

The goal of this project is to build a spam filter for sms messages using a Naive Bayes algorithm. The algorithm uses conditional probability to evaluate whether a new message is more likely to be spam or not. <br> 
The dataset is taken from the UCI Machine Learning repository and contains 5572 sms messages that are labeled as being spam or legitimate messages (ham).

In [1]:
import pandas as pd
# Read in dataset
sms_data = pd.read_csv('SMSSpamCollection', sep='\t', header = None, names=['Label', 'SMS'])

### Descriptive statistics

In [3]:
sms_data.describe()

,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [4]:
# Percent of spam vs. non-spam observations
round((sms_data['Label'].value_counts()/len(sms_data)) * 100, 2)

ham     86.59
spam    13.41
Name: Label, dtype: float64

### Split dataset in training and test data

In [5]:
# 80% train, 20% test data
train = sms_data.sample(frac=0.8,random_state=1)
test = sms_data.drop(train.index).sample(frac=1)

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [6]:
# Check spam vs. non-spam distribution in train and test data
print(train['Label'].value_counts()/len(train))

print(test['Label'].value_counts()/len(test))

ham     0.86541
spam    0.13459
Name: Label, dtype: float64
ham     0.868043
spam    0.131957
Name: Label, dtype: float64


Randomization did not significantly impact the distribution of the target variable

## Data cleaning

In [7]:
# Remove punctuation of sms messages
train['SMS'] = train['SMS'].str.replace(r'[^\w\s]+', '')

# Transform to lower case
train['SMS'] = train['SMS'].str.lower() 

### Create wide format with columns for word count

In [8]:
# Split SMS content by whitespace
train['SMS'] = train['SMS'].str.split()

# Create vocabulary list of unique words
vocabulary = []
for sms in train['SMS']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [9]:
# Create dictionary of unique word count per SMS
word_counts_per_sms = {unique_word: [0] * len(train['SMS']) for unique_word in vocabulary}
for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [10]:
# Transform in Dataframe and combine with sms_data
word_count_df = pd.DataFrame(word_counts_per_sms)
sms_data_count = pd.concat([sms_data, word_count_df])

## Set-up of Parameters

In [11]:
# Set-up parameters
n_spam = train['Label'].value_counts()['spam']
n_ham = train['Label'].value_counts()['ham']
n_vocabulary = len(vocabulary)
alpha = 1

p_spam = n_spam / len(train)
p_ham = n_ham / len(train)

In [12]:
# Count number of word mentions in spam and in ham df
spam_prob_dict = {unique_word: 0 for unique_word in vocabulary}
ham_prob_dict = {unique_word: 0 for unique_word in vocabulary}

spam_df = train.loc[train['Label'] == 'spam',]
spam_count = spam_df['SMS'].apply(lambda x: pd.Series(x).value_counts()).sum().sort_values()
ham_df = train.loc[train['Label'] == 'ham',]
ham_count = ham_df['SMS'].apply(lambda x: pd.Series(x).value_counts()).sum().sort_values()

<ipython-input-12-8dbfa9e127dd>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ham_count = ham_df['SMS'].apply(lambda x: pd.Series(x).value_counts()).sum().sort_values()


#### Calculate conditional probabilities
The probability for a message being spam given a certain word is calulated using Bayes' theorem:<br>
<img src="bayes_theorem.png" width="300" height="200"/>

In [13]:
# Calculate conditional probabilities
for word in vocabulary:
    #Conditional probability: P(word|spam)
    if word in spam_count:
        n_word_spam = spam_count[word]
    else:
        n_word_spam = 0
    p_word_spam = (n_word_spam + alpha) / (n_spam + alpha * n_vocabulary)
    spam_prob_dict[word] = p_word_spam
    
    #Conditional probability: P(word|ham)
    if word in ham_count:
        n_word_ham = ham_count[word]
    else:
        n_word_ham = 0
    p_word_ham = (n_word_ham + alpha) / (n_ham + alpha * n_vocabulary)
    ham_prob_dict[word] = p_word_ham

## Build Spam filter

The spam filter takes a new message as an input, calculates the conditional probability for the message to be spam and classifies it accordingly. <br>
The combining of individual word-spam probabilities per SMS is done with the following formula:<br>
<img src="prob_updating.png" width="400" height="200"/>


In [14]:
import re
def classify(message):
    
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    # Initial probability
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Update probability using probabilities for individual words
    for word in message:
        if word in spam_prob_dict:
            p_spam_given_message *= spam_prob_dict[word]
        if word in ham_prob_dict:
            p_ham_given_message *= ham_prob_dict[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

### Test Spam filter

In [15]:
test['predicted'] = test['SMS'].apply(classify)
test.head(10)

,Label,SMS,predicted
0,spam,Your weekly Cool-Mob tones are ready to downlo...,ham
1,ham,I (Career Tel) have added u as a contact on IN...,ham
2,ham,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C...,ham
3,ham,"Now u sound like manky scouse boy steve,like! ...",ham
4,ham,Sindu got job in birla soft ..,ham
5,ham,"Goodmorning, today i am late for 2hrs. Because...",ham
6,ham,thanks for the temales it was wonderful. Thank...,ham
7,ham,I went to project centre,ham
8,ham,Do you think i can move &lt;#&gt; in a week,ham
9,ham,Ok. But i finish at 6.,ham


In [16]:
test['predicted'].value_counts()/len(test)

ham     0.906643
spam    0.093357
Name: predicted, dtype: float64

#### Accuracy of the classification

In [17]:
total = len(test)
correct = len(test.loc[test['Label'] == test['predicted'],])
accuracy = correct/total
print(f'Accuracy: {round(accuracy,4)}')

Accuracy: 0.9596


#### Specificity & Sensitivity of the classifcation

In [20]:
true_pos = len(test.loc[(test['Label'] == 'spam') & (test['predicted'] == 'spam'),])
false_pos = len(test.loc[(test['Label'] == 'spam') & (test['predicted'] == 'ham'),])
true_neg = len(test.loc[(test['Label'] == 'ham') & (test['predicted'] == 'ham'),])
false_neg = len(test.loc[(test['Label'] == 'ham') & (test['predicted'] == 'spam'),])

sensitivity = true_pos / (true_pos + false_neg)
specificity = true_neg / (true_neg + false_pos)
print(f'Sensitivity: {round(sensitivity,4)}')
print(f'Specificity: {round(specificity,4)}')

Sensitivity: 0.9904
Specificity: 0.9564


## Discussion

#### Performance
The spam filter using a Naive Bayes algorithm performs quite good on the test set. It classifies over 95% of messages correctly and has a very high sensitivity of over 99%, thus detecting almost all spam messages. It does so, while still having a high specificity, correctly classifying over 95% of all non-spam messages.

#### Limitations
The spam filter is based on a rather small sample and would obviously benefit from a larger training dataset. The training data should ideally containg a large variety of different SMS sources to get a more representative picture of non-spam as well as spam messages. <br>
Naive Bayes spam filters are also susceptible to Bayesian poisoning - a technique where a spammer includes a large amount of legitimate text to improve the messages spam-score.